# The subprocess package

The subprocess package is part of the Python standard library, however, it has changed significantly between Python2 and Python3, so we will cover some of the Py3 specific code, but mostly run code that is compatible between both, since you will see the Py2 examples in other peoples code. The following link is the official Python documentation for subprocess. It is bit dense, but has extra information for troubleshooting https://docs.python.org/3/library/subprocess.html.

### Note for windows users
This notebook cannot be run on windows because it requires installing external software that is not available for windows (e.g., the programs `muscle` and `raxml`). However, you can run this notebook on the Habanero cluster by starting the notebook there. See instructions in notebook `nb-7.3-tunneling` to set this up. 

### Requried software
The following packages are required for this notebook. Install with conda and restart the notebook.

In [1]:
#conda install raxml -c bioconda
#conda install muscle -c bioconda
#conda install toytree -c eaton-lab

SyntaxError: invalid syntax (<ipython-input-1-29ef1da49942>, line 1)

In [1]:
import subprocess as sps
import toytree
import shlex
import sys

### What does subprocess do?

The subprocess module can be used to call bash commands or other shell scripts using pure Python code. In other words, under the hood when you run subprocess it opens up a terminal, runs your command, and returns the result as a string of text along with any accompanying error messages. 

### Why use subprocess?
One of the most powerful uses of Python is in creating **pipelines** programs, a program that calls other programs (sometimes many of them) and returns nicely formatted results. Python is really nice for this because it is such a good language for parsing text files (results), and because it is easily readable. Thus you can write a Python program that works as a pipeline to run a very complex bash script and format it to be easily readable and executable in Python. Below is a simple example that calls `ls -l` to list the files in our current directory. We'll explain the functions in greater detail soon. 


In [2]:
## write a bash command as a list arg and execute it using .communicate()
process = sps.Popen(["ls", "-l"], stdout=sps.PIPE)  
stdout, stderr = process.communicate()             

In [3]:
## stdout is the returned result, here decoded from bytes to string encoding
print(stdout.decode())                              

total 128
-rw-r--r-- 1 mv2640 user    82 Mar  7 17:10 filestderr
-rw-r--r-- 1 mv2640 user     6 Mar  7 17:10 filestdout
-rw-r--r-- 1 mv2640 user 24384 Mar  7 14:44 nb-7.1-ssh.ipynb
-rw-r--r-- 1 mv2640 user 61650 Mar  9 07:44 nb-7.2-subprocess.ipynb
-rw-r--r-- 1 mv2640 user 11387 Mar  7 14:44 nb-7.3-tunneling.ipynb



In [4]:
## check for errors; returncode 0 = no error; > 0 is some kind of error
process.returncode                     

0

In [5]:
## ask what was the command again?
process.args  

['ls', '-l']

### Blocking and asynchronous execution
When you send a subprocess to be run it is actually being executed on a separate thread from the one that is running Python. In essence, you are running *parallel code* when you run a subprocess. We'll talk much more about parallel code in later weeks. But this is our first introduction to the idea. The simplest code for running subprocess is to use the function `.communicate()` which will **block** your Python code from being able to execute any more commands until the subprocess is finished. This can be useful if your next piece of code depends on the results of the subprocess. However, other times we might be interested in doing something else while the subprocess is running, in that case we will run *asynchronous* code, meaning that we can continue calling Python code and query the code the object to ask whether it is finished or not. Below are examples of both. 

In [6]:
## send a job to run that takes 10 seonds, communicate() blocks until finished
proc = sps.Popen(["sleep", "10"])
stdout, stderr = proc.communicate()

In [7]:
## send a job to run but don't block, we'll query in the next cell whether it's ready
proc = sps.Popen(["sleep", "10"])


In [10]:
## does not block; asks whether proc is ready; when finished it returns the returncode
## execute this several times until you see a result returned.
proc.poll()

0

### Asynchronous execution
In the code below we use the Python library time to call the same type of `sleep` function that is being called in the subprocess using the `sleep` bash program. Here we use a while loop to continue executing Python code until the subprocess is finished. We use `.poll()` to query whether it is finished, and if it is then we call `break` to exit the while loop. If it is not finished then we tell the code to wait two seconds and print a short message. This is an easy way to create something like a progress bar for a program. 

In [11]:
import time

## send a job to run for 10 seconds
process = sps.Popen(["sleep", "10"])

## while statement means keep looping 
while 1:
    ## break exits the 'while' loop
    if process.poll() == 0:
        print("done")
        break
        
    ## not finished, print, sleep, and continue looping
    else:
        print("waiting two seconds")
        time.sleep(2)   

waiting two seconds
waiting two seconds
waiting two seconds
waiting two seconds
waiting two seconds
done


### stdout, stdin, and stderr
In our first session we touched briefly on the concepts of `stdin`, `stdout`, and `stderr` as they relate to some of the bash commands that we learned. But now we'll talk about them in more detail. These three terms refer to three distinct communication channels that programs uses to communicate. It is a standardized format that has been around since the early days of Unix.

The short description of these three channels is that `stdin` is the stream of data that is *entered in* to the program, whereas `stderr` and `stdout` are two channels that are returned as output from the program. Typically (though not always) `stdout` will contain results and `stderr` will be reserved for error messages. 

### stdout, stdin, and stderr in bash 
Below are some examples using `stdin`, `stdout`, or `stderr` differently in common bash scripts. Because the cell below has a `"%%bash"` header in it the code is executed as bash code instead of as IPython code. In bash these three channels can also be referred by the numbers 0, 1, and 2, respectively. Below I use the convention `1>` and `2>` to write to stdout and stderr. 

In [12]:
%%bash
## by default an echo statement returns stdout & stderr 
echo "hello world"

hello world


In [13]:
%%bash
## similarly, writing to a file combines stdout & stderr
echo "hello world" > filestdout
cat filestdout

hello world


In [14]:
%%bash
## explicitly write only stdout to a file
echo "hello world" | cut -b 1-5 1> filestdout
cat filestdout

hello


In [15]:
%%bash
## this command is invalid (has : instead of -) and raises an error
## which we write to filestderr. cat reads the file and prints it to stdout.
echo "hello world" | cut -b 1:5 2> filestderr
cat filestderr

cut: invalid byte, character or field list
Try 'cut --help' for more information.


### stdin, stdout, and stderr in Python 

The `sys` module can be used to print separately to stdout and stderr in Python. This can be done using the `write()` command from `sys` or by adding it as an argument to the print function (Python3 only). 

In [16]:
sys.stdout.write("hello")

hello

In [17]:
sys.stderr.write("hello")

hello

In [18]:
print("hello", file=sys.stdout)

hello


In [19]:
print("hello", file=sys.stderr)

hello


### Splitting a command line string into a list

The `shlex` library provides a convenient argument for splitting command strings into a list for use in subprocess. It differs in a few ways from the standard `split` command that can be used on strings, such as by recognizing when a space is part of a file name. We'll use it convert command line arguments to lists that can be entered to `subprocess`. 

In [20]:
cmd = "cat my\ file\ name\ with\ spaces\ in\ it"
shlex.split(cmd)

['cat', 'my file name with spaces in it']

In [21]:
cmd = "cat my file name with spaces in it"
shlex.split(cmd)

['cat', 'my', 'file', 'name', 'with', 'spaces', 'in', 'it']

### Calling a single command and parsing its result

The `Popen` class of subprocess is the most flexible way to call other programs, although there are a few other ways as well that you can find in the documentation. It takes as a required argument list of commands where each element in the list is a separate part of an argument that would normally be separated by a space in the command line string. The example below splits `"ls -l"` into `["ls", "-l"]`. It can then take additional arguments that tell it how to redirect the stdin, stderr, and stdout. These are important for directing output either to files, or to be parsed as a string directly. As you will see below we can pipe together multiple commands by using the special value `PIPE` to connect multiple subprocess commands. 

In [22]:
cmd = "ls -l"
cmdlist = shlex.split(cmd)
proc = sps.Popen(cmdlist, stdout=sps.PIPE)
stdout, stderr = proc.communicate()

In [23]:
print(stdout.decode())

total 128
-rw-r--r-- 1 mv2640 user    82 Mar  9 08:13 filestderr
-rw-r--r-- 1 mv2640 user     6 Mar  9 08:13 filestdout
-rw-r--r-- 1 mv2640 user 24384 Mar  7 14:44 nb-7.1-ssh.ipynb
-rw-r--r-- 1 mv2640 user 61946 Mar  9 08:14 nb-7.2-subprocess.ipynb
-rw-r--r-- 1 mv2640 user 11387 Mar  7 14:44 nb-7.3-tunneling.ipynb



### Using a PIPE to connect multiple program calls

It is a bit more complex to call multiple programs that pipe results into one another, but using `subprocess` this can be done in a very clean way. Here we will call two programs where one pipes results to another. 

```bash
echo "hello world" | cut -b 1-5
```

In [24]:
## write the commands to execute
cmd1 = "echo hello world"
cmd2 = "cut -b 1-5"

## create subprocesses for each, designating stdin and stdout connected by pipes
proc1 = sps.Popen(shlex.split(cmd1), stdout=sps.PIPE)
proc2 = sps.Popen(shlex.split(cmd2), stdin=proc1.stdout, stdout=sps.PIPE)

## execute just the last object, the former will inherit execution
stdout, stderr = proc2.communicate()

In [25]:
## print result nicely
print(stdout.decode())

hello



### Call some more interesting biological programs

#### Muscle
Below we will call the two programs we installed at the top of this notebook. The first is a sequence alignment program called `muscle`. It takes as input a file with genetic sequence data in `fasta` format and uses an algorithm to find the optimal alignment of the sequences (the alignment that minimized the number of differences among them.) This type of operation is commonly used when doing phylogenetic analyses to ensure that homologous sites are being compared when searching for genetic substitutions. Muscle uses the following syntax: 

```
muscle -in <fasta file> -out <fasta file>
```

However, it is also written so that if no `-out` argument is provided then it simply pipes its output to `stdout`, and similarly if no `-in` argument is provided then it looks for input on `stdin`. Let's try piping input to this program as a string using `stdin`. This can save us the trouble of having to write an input or output file, since we can just pipe string data into the program, and catch the stdout as a string. That is much more efficient. First I'll show the normal way of running `muscle` with file inputs and then the better way of using pipes. 

#### the normal way of running muscle with file input and outputs

In [26]:
## fasta data as a string
fasta_data = """\
>sample1
AAGGCCTTAAGGCGTTAAAACCTTAAGGCCTTAAGGCCTT
>sample2
AAGGCCTTTTGGCGTTAAAACCTTAAGGCCTAAAGGCCTT
>sample3
AAGCCCTAAAGCCCTTAAGGCCAAGGCCATAAGGCCGTGG
>sample4
AAGGCCTAAAGGCCTTAAGGCCAAGGCCATAAGGCCGTCG
>sample5
TTGGCCTATAGGCCTTTAGGCCAAGGCCTTGACGCCTAAG
"""

## write the string to a file
with open("example.fa", 'w') as out:
    out.write(fasta_data)

In [27]:
## run the full command with stdout written to a file
cmd = "muscle -in ./example.fa -out ./aligned.fa"
proc = sps.Popen(shlex.split(cmd), stderr=sps.PIPE)  # log is written to stderr 
stdout, stderr = proc.communicate()

In [28]:
## view the logged stats for the alignment
print(stderr.decode())


MUSCLE v3.8.1551 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

example 5 seqs, lengths min 40, max 40, avg 40
00:00:00     24 MB(1%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00     24 MB(1%)  Iter   1  100.00%  K-mer dist pass 2
00:00:00     25 MB(1%)  Iter   1  100.00%  Align node
00:00:00     25 MB(1%)  Iter   1  100.00%  Root alignment
00:00:00     25 MB(1%)  Iter   2  100.00%  Root alignment
00:00:00     25 MB(1%)  Iter   3  100.00%  Refine biparts



In [29]:
## read the output from a file
with open("aligned.fa", 'r') as infile:
    print(infile.read())

>sample5
TTGGCCTATAGGCCTTTAGGCC--AAGGCCTTGACGCCTAAG
>sample3
AAGCCCTAAAGCCCTTAAGGCC--AAGGCCATAAGGCCGTGG
>sample4
AAGGCCTAAAGGCCTTAAGGCC--AAGGCCATAAGGCCGTCG
>sample1
AAGGCCTTAAGGCGTTAAAACCTTAAGGCCTTAAGGCCTT--
>sample2
AAGGCCTTTTGGCGTTAAAACCTTAAGGCCTAAAGGCCTT--



### Alternative is to use PIPEs directly
This way, as I said, can make for much cleaner and faster code. 

In [31]:
## pipe stdin, stderr, and stdout. The stdin is passed during communicate.
proc = sps.Popen(["muscle"], stdin=sps.PIPE, stdout=sps.PIPE, stderr=sps.PIPE)
stdout, stderr = proc.communicate(input=fasta_data.encode())

In [32]:
## view the stdout variable
print(stdout.decode())

>sample5
TTGGCCTATAGGCCTTTAGGCC--AAGGCCTTGACGCCTAAG
>sample3
AAGCCCTAAAGCCCTTAAGGCC--AAGGCCATAAGGCCGTGG
>sample4
AAGGCCTAAAGGCCTTAAGGCC--AAGGCCATAAGGCCGTCG
>sample1
AAGGCCTTAAGGCGTTAAAACCTTAAGGCCTTAAGGCCTT--
>sample2
AAGGCCTTTTGGCGTTAAAACCTTAAGGCCTAAAGGCCTT--



In [33]:
## view the stderr variable
print(stderr.decode())


MUSCLE v3.8.1551 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

- 5 seqs, lengths min 40, max 40, avg 40
00:00:00     23 MB(1%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00     23 MB(1%)  Iter   1  100.00%  K-mer dist pass 2
00:00:00     24 MB(1%)  Iter   1  100.00%  Align node
00:00:00     24 MB(1%)  Iter   1  100.00%  Root alignment
00:00:00     24 MB(1%)  Iter   2  100.00%  Root alignment
00:00:00     24 MB(1%)  Iter   3  100.00%  Refine biparts



### Another command line program: RAxML
Another program we will test out is `raxml`, this is a CLI program for inferring phylogenetic trees from sequence alignments. Here we are going to write our first pipeline program by parsing the results of a `muscle` execution and then reformatting those results using Python before entering them into `raxml`. The fasta sequence alignment format can be seen above. In it, each sequence has a line for the name which starts with ">" and then the following line has the sequence. Raxml also supporta a different sequence format called "phylip", in which the names and sequences are on the same line, and there is no ">" at the beginning of names. Let's write a short converter to change from fasta to phylip.  

In [34]:
fasta_string = stdout.decode()
fasta_string

'>sample5\nTTGGCCTATAGGCCTTTAGGCC--AAGGCCTTGACGCCTAAG\n>sample3\nAAGCCCTAAAGCCCTTAAGGCC--AAGGCCATAAGGCCGTGG\n>sample4\nAAGGCCTAAAGGCCTTAAGGCC--AAGGCCATAAGGCCGTCG\n>sample1\nAAGGCCTTAAGGCGTTAAAACCTTAAGGCCTTAAGGCCTT--\n>sample2\nAAGGCCTTTTGGCGTTAAAACCTTAAGGCCTAAAGGCCTT--\n'

### converter
Examine the fasta_string above and the code below to understand how it parses this string into phylip format. Run some of the split commands on the fasta_string separately to see how they split it. 


In [35]:
def fasta_to_phylip(fasta_string):
    ntaxa = len(fasta_string.split(">")[1:])
    seqlen = len(fasta_string.split("\n")[1])
    seqs = [i.strip().replace("\n", "   ") for i in fasta_string.split(">")[1:]]
    seqstring = "\n".join(seqs)
    phylip = "{} {}\n{}".format(ntaxa, seqlen, seqstring)
    return phylip

In [36]:
phylip_string = fasta_to_phylip(fasta_string)
print(phylip_string)

5 42
sample5   TTGGCCTATAGGCCTTTAGGCC--AAGGCCTTGACGCCTAAG
sample3   AAGCCCTAAAGCCCTTAAGGCC--AAGGCCATAAGGCCGTGG
sample4   AAGGCCTAAAGGCCTTAAGGCC--AAGGCCATAAGGCCGTCG
sample1   AAGGCCTTAAGGCGTTAAAACCTTAAGGCCTTAAGGCCTT--
sample2   AAGGCCTTTTGGCGTTAAAACCTTAAGGCCTAAAGGCCTT--


### Run raxml

Raxml doesn't allow input through `stdin` so we will have to write the data to a file. All programs are written differently, so you often need to examine the documentation or help command to find out how the input and outputs are used. In addition, raxml writes it's log information to stdout instead of stderr. Again, this is just something that some developers do. Because the output is always written to a file for raxml it doesn't really matter whether the log is written to stderr or stdout. Here we use the required argument `-n` to give a name to the output file. If an output file with that name already exists then it will raise an error. 


The program raxml takes a lot of arguments which I won't describe for now, but these are the minimal number of arguments required to infer a tree using the algorithm designated. We will discuss some of the detail of phylogenetic inference in a later session. For now, its enough to know that the following program will take the aligned sequence data and infer the best phylogenetic tree based on the observed differences among sequences. 

In [37]:
## create the command string
cmd = "raxmlHPC -f a -m GTRGAMMA -p 123 -x 123 -N 10 -s aligned.phy -n out10"
proc = sps.Popen(shlex.split(cmd), stdout=sps.PIPE, stderr=sps.PIPE)

## align the data and write alignment as a phylip file
with open("aligned.phy", 'w') as out:
    out.write(phylip_string)

## run the job
stdout, stderr = proc.communicate()

In [38]:
print(stdout.decode())



Using BFGS method to optimize GTR rate parameters, to disable this specify "--no-bfgs" 



This is RAxML version 8.2.10 released by Alexandros Stamatakis on March 2017.

With greatly appreciated code contributions by:
Andre Aberer      (HITS)
Simon Berger      (HITS)
Alexey Kozlov     (HITS)
Kassian Kobert    (HITS)
David Dao         (KIT and HITS)
Sarah Lutteropp   (KIT and HITS)
Nick Pattengale   (Sandia)
Wayne Pfeiffer    (SDSC)
Akifumi S. Tanabe (NRIFS)
Charlie Taylor    (UF)


Alignment has 20 distinct alignment patterns

Proportion of gaps and completely undetermined characters in this alignment: 4.76%

RAxML rapid bootstrapping and subsequent ML search

Using 1 distinct models/data partitions with joint branch length optimization



Executing 10 rapid bootstrap inferences and thereafter a thorough ML search 

All free model parameters will be estimated by RAxML
GAMMA model of rate heterogeneity, ML estimate of alpha-parameter

GAMMA Model parameters will be estimated up to an 

### We could wrap this all into a Class object
Finish writing the class object. Write it so that it can accomplish the following tasks: 

+ all of the attribute variables in `__init__` (e.g., self.aligned) are filled by functions called during the `.run()` function.
+ the `_run_raxml()` function removes an existing output file with the same name if it exists.
+ the `_run_raxml()` function modifies the command for raxml by replacing the argument `-n out` with `-n <outname>`, and then running it with subprocess and parsing the result.
+ the `_run_raxml()` function parses the result from the raxml output file `RAxML_bestTree.<outname>` as a string and returns it.

In [61]:
class Phylogeny:
    def __init__(self, fasta_string):
        ## store data
        self.fasta = fasta_string
        self.aligned = None
        self.phylip = None
        self.tree = None
        self.log = None
        
    # private functions
    def _muscle_align(self): 
        proc = sps.Popen(["muscle"], stdin=sps.PIPE, stdout=sps.PIPE, stderr=sps.PIPE)
        stdout, stderr = proc.communicate(input=fasta_data.encode())
        self.log = (stderr.decode())
        return self.log
    
    def _aligned_fasta_to_phylip(self):
        ntaxa = len(self.log.split(">")[1:])
        seqlen = len(self.log.split("\n")[1])
        seqs = [i.strip().replace("\n", "   ") for i in self.log.split(">")[1:]]
        seqstring = "\n".join(seqs)
        self.phylip = "{} {}\n{}".format(ntaxa, seqlen, seqstring)
        return(self.phylip)
    
    def _run_raxml(self, outname):
        cmd = "raxmlHPC -f a -m GTRGAMMA -p 123 -x 123 -N 10 -s aligned.phy -n out11"#changed to out11
        proc = sps.Popen(shlex.split(cmd), stdout=sps.PIPE, stderr=sps.PIPE)
        with open("aligned.phy", 'w') as out:
            out.write(phylip_string)
        stdout, stderr = proc.communicate()
        self.tree = (stdout.decode())
        return self.tree
    
    # public function
    def run(self, outname):
        self.aligned = self._muscle_align()
        self.phylip = self._aligned_fasta_to_phylip()
        self.tree = self._run_raxml(outname)

### Test the code

In [62]:
## run your code
p = Phylogeny(fasta_string)
p.run(outname="test")
print(p.tree)



Using BFGS method to optimize GTR rate parameters, to disable this specify "--no-bfgs" 



This is RAxML version 8.2.10 released by Alexandros Stamatakis on March 2017.

With greatly appreciated code contributions by:
Andre Aberer      (HITS)
Simon Berger      (HITS)
Alexey Kozlov     (HITS)
Kassian Kobert    (HITS)
David Dao         (KIT and HITS)
Sarah Lutteropp   (KIT and HITS)
Nick Pattengale   (Sandia)
Wayne Pfeiffer    (SDSC)
Akifumi S. Tanabe (NRIFS)
Charlie Taylor    (UF)


Alignment has 20 distinct alignment patterns

Proportion of gaps and completely undetermined characters in this alignment: 4.76%

RAxML rapid bootstrapping and subsequent ML search

Using 1 distinct models/data partitions with joint branch length optimization



Executing 10 rapid bootstrap inferences and thereafter a thorough ML search 

All free model parameters will be estimated by RAxML
GAMMA model of rate heterogeneity, ML estimate of alpha-parameter

GAMMA Model parameters will be estimated up to an 

In [63]:
## plot the tree
p.tree = "RAxML_bestTree.out10"
tree = toytree.tree(p.tree)
tree.draw(width=300);

sample4 sample3 sample2 sample1 sample5